In [ ]:
%reset -f
import pandas as pd
import numpy as np
import scipy.stats as stats
from pathlib import Path
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# --- CONFIGURATION ---
BASE_PATH = Path("./Documents/airr-ml/data/train_datasets/train_dataset_3/")
METADATA_PATH = BASE_PATH / "metadata.csv"
SEED = 42

# Set random seed for reproducibility
np.random.seed(SEED)

# Lowered thresholds slightly for safety, but kept strict enough to be useful
MIN_POS_PATIENTS = 3   # Lowered from 7 to 3
MAX_NEG_PATIENTS = 0   # Keep strict: signal shouldn't be in negatives
P_VALUE_CUTOFF = 0.05 # Relaxed from 1e-5 to 1e-4

# Load Metadata & Split with FIXED SEED
df_meta = pd.read_csv(METADATA_PATH)
train_df, val_df = train_test_split(
    df_meta, test_size=0.2, stratify=df_meta['label_positive'], random_state=SEED
)

print(f"Split: {len(train_df)} train, {len(val_df)} validation (seed={SEED})")

def load_signatures(filename):
    path = BASE_PATH / filename
    try:
        df = pd.read_csv(path, sep='\t', usecols=['junction_aa', 'v_call', 'j_call'])
        return set(zip(df['junction_aa'], df['v_call'], df['j_call']))
    except:
        return set()

# --- COUNTING ---
pos_train = train_df[train_df['label_positive'] == True]['filename'].tolist()
neg_train = train_df[train_df['label_positive'] == False]['filename'].tolist()
n_pos = len(pos_train)
n_neg = len(neg_train)

print(f"Analyzing {n_pos} Positive Patients and {n_neg} Negative Patients.")

counts_dict = {}

print("Counting sequences...")
for fname in tqdm(pos_train, desc="Positives"):
    for sig in load_signatures(fname):
        if sig not in counts_dict: counts_dict[sig] = [0, 0]
        counts_dict[sig][0] += 1

# Optimization: Only count negatives if the sequence was seen in positives
for fname in tqdm(neg_train, desc="Negatives"):
    for sig in load_signatures(fname):
        if sig in counts_dict:
            counts_dict[sig][1] += 1

# --- DIAGNOSTIC FISHER'S TEST ---
significant_features = []
print(f"\nScanning for features (Min Pos={MIN_POS_PATIENTS}, Max Neg={MAX_NEG_PATIENTS}, P < {P_VALUE_CUTOFF})...")

# DIAGNOSTIC COUNTERS
stats_counter = {
    "total_checked": 0,
    "failed_min_pos": 0,
    "failed_max_neg": 0,
    "failed_p_value": 0,
    "passed": 0
}

for sig, counts in tqdm(counts_dict.items()):
    stats_counter["total_checked"] += 1
    pos_with = counts[0]
    neg_with = counts[1]
    
    # Check 1: Frequency Filter
    if pos_with < MIN_POS_PATIENTS:
        stats_counter["failed_min_pos"] += 1
        continue
        
    # Check 2: Safety Filter (Anti-Noise)
    if neg_with > MAX_NEG_PATIENTS: 
        stats_counter["failed_max_neg"] += 1
        continue

    pos_without = n_pos - pos_with
    neg_without = n_neg - neg_with
    
    _, p_val = stats.fisher_exact(
        [[pos_with, neg_with], [pos_without, neg_without]], 
        alternative='greater'
    )
    
    # Check 3: Statistical Significance
    if p_val >= P_VALUE_CUTOFF:
        stats_counter["failed_p_value"] += 1
        continue
        
    # PASSED ALL CHECKS
    stats_counter["passed"] += 1
    significant_features.append({
        'signature': sig,
        'p_value': p_val,
        'pos_count': pos_with,
        'neg_count': neg_with
    })

# --- REPORTING RESULTS ---
print("\n" + "="*40)
print("DIAGNOSTIC REPORT")
print(f"Total Unique Sequences Seen: {stats_counter['total_checked']}")
print(f"Discarded (Too Rare in Positives): {stats_counter['failed_min_pos']}")
print(f"Discarded (Too Common in Negatives): {stats_counter['failed_max_neg']}")
print(f"Discarded (Not Significant): {stats_counter['failed_p_value']}")
print(f"KEPT (Significant Signals): {stats_counter['passed']}")
print("="*40)

if len(significant_features) == 0:
    print("ERROR: No features found. Please RELAX your thresholds.")
    print("Try lowering MIN_POS_PATIENTS to 3 or 4.")
else:
    enrichment_df = pd.DataFrame(significant_features).sort_values('p_value')
    print("Top 5 Sequences found:")
    print(enrichment_df.head(5)[['pos_count', 'neg_count', 'p_value']])
    
    enrichment_df.to_pickle("enriched_signatures_strict.pkl")
    print("\nSaved to 'enriched_signatures_strict.pkl'. You can now run Part 2.")

Split: 320 train, 80 validation (seed=42)
Analyzing 160 Positive Patients and 160 Negative Patients.
Counting sequences...


Negatives: 100%|██████████| 160/160 [00:03<00:00, 40.72it/s]



Scanning for features (Min Pos=3, Max Neg=0, P < 0.05)...


100%|██████████| 3357656/3357656 [00:02<00:00, 1127270.06it/s]



DIAGNOSTIC REPORT
Total Unique Sequences Seen: 3357656
Discarded (Too Rare in Positives): 3297023
Discarded (Too Common in Negatives): 44455
Discarded (Not Significant): 15449
KEPT (Significant Signals): 729
Top 5 Sequences found:
     pos_count  neg_count   p_value
35          10          0  0.000845
118         10          0  0.000845
287         10          0  0.000845
158         10          0  0.000845
90           9          0  0.001740

Saved to 'enriched_signatures_strict.pkl'. You can now run Part 2.


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from scipy import sparse

# --- CONFIGURATION ---
BASE_PATH = Path("./Documents/airr-ml/data/train_datasets/train_dataset_3/")
METADATA_PATH = BASE_PATH / "metadata.csv"
SEED = 42

# Set random seed for reproducibility
np.random.seed(SEED)

# Load Metadata & Split (SAME seed as Cell 1)
df_meta = pd.read_csv(METADATA_PATH)
train_df, val_df = train_test_split(
    df_meta, 
    test_size=0.2, 
    stratify=df_meta['label_positive'], 
    random_state=SEED
)

# Load your 897 winning signatures
enrichment_df = pd.read_pickle("enriched_signatures_strict.pkl")
feature_list = enrichment_df['signature'].tolist()
feature_map = {sig: i for i, sig in enumerate(feature_list)}

print(f"Loaded {len(feature_list)} features.")

def load_signatures(filename):
    path = BASE_PATH / filename
    try:
        df = pd.read_csv(path, sep='\t', usecols=['junction_aa', 'v_call', 'j_call'])
        return set(zip(df['junction_aa'], df['v_call'], df['j_call']))
    except:
        return set()

def create_sparse_matrix(filenames, feature_map):
    """
    Creates a (Samples x Features) binary matrix.
    Rows = Patients, Cols = Specific Public Sequences.
    """
    n_samples = len(filenames)
    n_features = len(feature_map)
    
    # We use a LIL matrix for efficient construction, then convert to CSR
    X = sparse.lil_matrix((n_samples, n_features), dtype=np.int8)
    
    for row_idx, fname in enumerate(tqdm(filenames, desc="Building Matrix")):
        sigs = load_signatures(fname)
        for s in sigs:
            if s in feature_map:
                col_idx = feature_map[s]
                X[row_idx, col_idx] = 1
                
    return X.tocsr()

# --- BUILD MATRICES ---
print("Building Training Matrix...")
X_train = create_sparse_matrix(train_df['filename'], feature_map)
y_train = train_df['label_positive'].astype(int).values

print("Building Validation Matrix...")
X_val = create_sparse_matrix(val_df['filename'], feature_map)
y_val = val_df['label_positive'].astype(int).values

# --- TRAIN LASSO MODEL ---
# penalty='l1' selects only the useful features
# C=1.0 is standard; if accuracy is low, try C=0.1 or C=10
print("Training Lasso (L1) Model...")
clf = LogisticRegression(
    penalty='l1', 
    solver='liblinear', 
    class_weight='balanced', 
    C=1.0, 
    random_state=SEED
)
clf.fit(X_train, y_train)

# --- EVALUATE ---
val_probs = clf.predict_proba(X_val)[:, 1]
val_auc = roc_auc_score(y_val, val_probs)
val_acc = accuracy_score(y_val, clf.predict(X_val))

# Count how many features the model actually used (non-zero weights)
n_active_features = np.sum(clf.coef_ != 0)

print("\n" + "="*30)
print(f"LASSO RESULTS (Features={len(feature_list)})")
print(f"Validation AUC: {val_auc:.4f}")
print(f"Validation Acc: {val_acc:.4f}")
print(f"Active Features Used: {n_active_features}")
print("="*30)

# DEBUG: Check the "Score" distribution
# If the model works, Positives should have high scores, Negatives low.
pos_scores = val_probs[y_val == 1]
neg_scores = val_probs[y_val == 0]
print(f"Avg Prob for Positives: {np.mean(pos_scores):.4f}")
print(f"Avg Prob for Negatives: {np.mean(neg_scores):.4f}")

Loaded 729 features.
Building Training Matrix...


Building Matrix: 100%|██████████| 320/320 [00:05<00:00, 57.18it/s]


Building Validation Matrix...


Building Matrix: 100%|██████████| 80/80 [00:01<00:00, 58.31it/s]

Training Lasso (L1) Model...

LASSO RESULTS (Features=729)
Validation AUC: 0.7719
Validation Acc: 0.6875
Active Features Used: 135
Avg Prob for Positives: 0.5408
Avg Prob for Negatives: 0.2847


In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from pathlib import Path
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from scipy import sparse
import glob
import os

# ==========================================
# --- 1. MASTER CONFIGURATION ---
# ==========================================
# Change this to 1, 2, or 3 to switch datasets
DATASET_NUM = 3 

# Paths
BASE_DIR = Path(f"./Documents/airr-ml/data")
TRAIN_DIR = BASE_DIR / f"train_datasets/train_dataset_{DATASET_NUM}/"
TEST_DIR = BASE_DIR / f"test_datasets/test_dataset_{DATASET_NUM}/"
METADATA_PATH = TRAIN_DIR / "metadata.csv"

# --- PARAMETER TUNING CHEAT SHEET ---
# Dataset 3 (Easy): MinPos=5, MaxNeg=1, P=0.01 (or strict MinPos=3, MaxNeg=0, P=0.05)
# Dataset 2 (Med):  MinPos=3, MaxNeg=0, P=0.20
# Dataset 1 (Hard): MinPos=2, MaxNeg=0, P=1.0 (Reliance on Lasso)

if DATASET_NUM == 3:
    MIN_POS_PATIENTS = 3
    MAX_NEG_PATIENTS = 0
    P_VALUE_CUTOFF = 0.05
elif DATASET_NUM == 2:
    MIN_POS_PATIENTS = 3
    MAX_NEG_PATIENTS = 0
    P_VALUE_CUTOFF = 0.20
elif DATASET_NUM == 1:
    MIN_POS_PATIENTS = 2
    MAX_NEG_PATIENTS = 0
    P_VALUE_CUTOFF = 1.0
# ==========================================

print(f"--- RUNNING PIPELINE FOR DATASET {DATASET_NUM} ---")
print(f"Params: MinPos={MIN_POS_PATIENTS}, MaxNeg={MAX_NEG_PATIENTS}, P={P_VALUE_CUTOFF}")

def load_signatures(filepath):
    """Efficiently loads unique (Junction, V, J) tuples"""
    try:
        df = pd.read_csv(filepath, sep='\t', usecols=['junction_aa', 'v_call', 'j_call'])
        return set(zip(df['junction_aa'], df['v_call'], df['j_call']))
    except Exception as e:
        print(f"Error reading {filepath}: {e}")
        return set()

# --- STEP 1: LOAD FULL METADATA ---
print("Step 1: Loading Metadata...")
df_meta = pd.read_csv(METADATA_PATH)
pos_files = df_meta[df_meta['label_positive'] == True]['filename'].tolist()
neg_files = df_meta[df_meta['label_positive'] == False]['filename'].tolist()

print(f"Training on FULL DATA: {len(pos_files)} Positives, {len(neg_files)} Negatives")

# --- STEP 2: FEATURE ENRICHMENT (FULL DATA) ---
print("Step 2: Counting Sequences (Enrichment)...")
counts_dict = {}

# Count Positives
for fname in tqdm(pos_files, desc="Positives"):
    for sig in load_signatures(TRAIN_DIR / fname):
        if sig not in counts_dict: counts_dict[sig] = [0, 0]
        counts_dict[sig][0] += 1

# Count Negatives (Optimization: Only check if seen in positives)
for fname in tqdm(neg_files, desc="Negatives"):
    for sig in load_signatures(TRAIN_DIR / fname):
        if sig in counts_dict:
            counts_dict[sig][1] += 1

# Perform Fisher's Test
print("Step 3: Calculating Statistics & Selecting Features...")
significant_features = []
n_pos = len(pos_files)
n_neg = len(neg_files)

for sig, counts in tqdm(counts_dict.items()):
    pos_with = counts[0]
    neg_with = counts[1]
    
    # FILTER 1: Frequency
    if pos_with < MIN_POS_PATIENTS: continue
    
    # FILTER 2: Exclusivity (Safety)
    if neg_with > MAX_NEG_PATIENTS: continue
    
    # FILTER 3: P-Value
    pos_without = n_pos - pos_with
    neg_without = n_neg - neg_with
    _, p_val = stats.fisher_exact(
        [[pos_with, neg_with], [pos_without, neg_without]], 
        alternative='greater'
    )
    
    if p_val < P_VALUE_CUTOFF:
        significant_features.append(sig)

# Create Feature Map
feature_list = significant_features
feature_map = {sig: i for i, sig in enumerate(feature_list)}
print(f"\nSELECTED FEATURES: {len(feature_list)}")

# Save the signals (Task 2 Requirement)
df_signals = pd.DataFrame(feature_list, columns=['junction_aa', 'v_call', 'j_call'])
df_signals.to_csv(f"dataset_{DATASET_NUM}_top_signals.csv", index=False)
print(f"Signals saved to 'dataset_{DATASET_NUM}_top_signals.csv'")

# --- STEP 4: BUILD TRAINING MATRIX ---
print("Step 4: Building Sparse Training Matrix...")
def create_sparse_matrix(filenames, base_dir):
    n_samples = len(filenames)
    n_features = len(feature_map)
    X = sparse.lil_matrix((n_samples, n_features), dtype=np.int8)
    
    for row_idx, fname in enumerate(tqdm(filenames)):
        sigs = load_signatures(base_dir / fname)
        for s in sigs:
            if s in feature_map:
                X[row_idx, feature_map[s]] = 1
    return X.tocsr()

X_train = create_sparse_matrix(df_meta['filename'], TRAIN_DIR)
y_train = df_meta['label_positive'].astype(int).values

# --- STEP 5: TRAIN FINAL MODEL ---
print("Step 5: Training Lasso Logistic Regression...")
# Use C=1.0 as default. If D1/D2 are noisy, lower C to 0.1
clf = LogisticRegression(
    penalty='l1', 
    solver='liblinear', 
    class_weight='balanced', 
    C=1.0, 
    random_state=42
)
clf.fit(X_train, y_train)
print(f"Model Trained. Active Features: {np.sum(clf.coef_ != 0)}")

# --- STEP 6: PREDICT ON TEST SET ---
print("Step 6: Processing Test Data...")
test_files = sorted([f.name for f in TEST_DIR.glob("*.tsv")])
if len(test_files) == 0:
    print("WARNING: No test files found! Check TEST_DIR path.")
else:
    print(f"Found {len(test_files)} test files.")
    
    # Build Test Matrix (Using same feature map)
    X_test = create_sparse_matrix(test_files, TEST_DIR)
    
    # Predict
    probs = clf.predict_proba(X_test)[:, 1]
    
    # Create Submission (strip .tsv extension)
    submission = pd.DataFrame({
        'filename': [f.replace('.tsv', '') for f in test_files],
        'probability': probs
    })
    
    sub_filename = f"dataset_{DATASET_NUM}_submission.csv"
    submission.to_csv(sub_filename, index=False)
    print(f"\nSUCCESS! Predictions saved to '{sub_filename}'")
    print(submission.head())

--- RUNNING PIPELINE FOR DATASET 3 ---
Params: MinPos=3, MaxNeg=0, P=0.05
Step 1: Loading Metadata...
Training on FULL DATA: 200 Positives, 200 Negatives
Step 2: Counting Sequences (Enrichment)...


Negatives: 100%|██████████| 200/200 [00:05<00:00, 39.44it/s]


Step 3: Calculating Statistics & Selecting Features...


100%|██████████| 4131240/4131240 [00:03<00:00, 1173544.40it/s]



SELECTED FEATURES: 1008
Signals saved to 'dataset_3_top_signals.csv'
Step 4: Building Sparse Training Matrix...


100%|██████████| 400/400 [00:07<00:00, 56.02it/s]


Step 5: Training Lasso Logistic Regression...
Model Trained. Active Features: 155
Step 6: Processing Test Data...
Found 400 test files.


100%|██████████| 400/400 [00:07<00:00, 56.04it/s]


SUCCESS! Predictions saved to 'dataset_3_submission.csv'
                           filename  probability
0  00bf3e8b5c44db7f9765a2c896308ca8     0.151802
1  00c0a5b0b5780839679ddcede9da56df     0.236472
2  00d82591a1ef87f94bc28b47f9a6c9a2     0.632017
3  016b2ba917244113cdc4dd790c5ff3c7     0.685355
4  02dd6983048af267517bc0a57d53e1b0     0.967874


In [8]:
submission

,filename,probability
0,00bf3e8b5c44db7f9765a2c896308ca8,0.151802
1,00c0a5b0b5780839679ddcede9da56df,0.236472
2,00d82591a1ef87f94bc28b47f9a6c9a2,0.632017
3,016b2ba917244113cdc4dd790c5ff3c7,0.685355
4,02dd6983048af267517bc0a57d53e1b0,0.967874
...,...,...
395,fe47d786705b124ee75e76dd970e7290,0.151802
396,feeae51e9d8974bc5048175d2300cfac,0.522939
397,fef053aa6ddabf5d3c7f4c18f9b08a99,0.151802
398,ff42cbc3605d26e8905cb98f19986321,0.202817
